In [ ]:
def create_model(optimizer='adam', dropout_rate=0.5):

    model = keras.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),

        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(dropout_rate),
        layers.Dense(62, activation='softmax')

    ])

    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(model=create_model, model__optimizer='adam', model__dropout_rate=0.5, epochs=10, batch_size=64, verbose=True)

param_grid = {
    'model__optimizer': ['adam', 'sgd', 'rmsprop'],
    'model__dropout_rate': [0.3, 0.5, 0.7],
    'batch_size': [32, 64, 128]
}

grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=4, verbose=10)

In [ ]:
grid_result = grid.fit(X, y)

In [ ]:
 print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

In [ ]:
import seaborn as sns

# Convert results to DataFrame
results = pd.DataFrame(grid.cv_results_)

# Extract relevant data
results = results[['param_model__dropout_rate', 'param_batch_size', 'mean_test_score']]
results['param_model__dropout_rate'] = results['param_model__dropout_rate'].astype(float)
results['param_batch_size'] = results['param_batch_size'].astype(int)

# Pivot table for heatmap
pivot_table = results.pivot(index="param_model__dropout_rate",
                            columns="param_batch_size",
                            values="mean_test_score")

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap="viridis", cbar_kws={'label': 'Mean Test Score'})
plt.title("GridSearchCV Results")
plt.xlabel("Batch Size")
plt.ylabel("Dropout Rate")
plt.show()